In [1]:
!pip install pyodbc psycopg2-binary sqlalchemy pyyaml pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datetime import date, datetime, timedelta
import pandas as pd
import yaml
from sqlalchemy import create_engine
import os



In [3]:
# database connections
with open('../config_fill.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_aw = config['Adventure_Works']
    config_etl = config['ETL_PRO']

url_aw = f"mssql+pyodbc://@{config_aw['host']}/{config_aw['dbname']}?driver={config_aw['driver'].replace(' ', '+')}&trusted_connection={config_aw['trusted_connection']}"
url_etl = f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}"

aw_engine = create_engine(url_aw)
etl_engine = create_engine(url_etl)

In [6]:
query_dates = """
SELECT 
    MIN(OrderDate) AS min_date,
    MAX(OrderDate) AS max_date
FROM Sales.SalesOrderHeader;
"""
df_dates = pd.read_sql(query_dates, aw_engine)
df_dates


,min_date,max_date
0,2011-05-31,2014-06-30


In [7]:
start_date = df_dates.loc[0, "min_date"]
end_date = df_dates.loc[0, "max_date"]

calendar = pd.date_range(start=start_date, end=end_date, freq='D')

df_dim_date = pd.DataFrame({"full_date": calendar})


In [8]:
df_dim_date["date_key"] = df_dim_date["full_date"].dt.strftime("%Y%m%d").astype(int)
df_dim_date["year"] = df_dim_date["full_date"].dt.year
df_dim_date["month"] = df_dim_date["full_date"].dt.month
df_dim_date["month_name"] = df_dim_date["full_date"].dt.month_name()
df_dim_date["quarter"] = df_dim_date["full_date"].dt.quarter
df_dim_date["day"] = df_dim_date["full_date"].dt.day
df_dim_date["day_of_week"] = df_dim_date["full_date"].dt.day_name()
df_dim_date["week_of_year"] = df_dim_date["full_date"].dt.isocalendar().week.astype(int)


In [9]:
df_dim_date.head()
df_dim_date.tail()
df_dim_date.shape


(1127, 9)

In [10]:
df_dim_date.to_sql(
    "dim_date",
    etl_engine,
    if_exists="replace",
    index=False
)


127